# Data Extraction Notebook for PV: Solar production on south- vs. west-facing rooftop and how solar systems align with 4CP events in Texas

## This notebook will connect to the database and extract the data live and put it into compressed zip files in this directory. 

This notebook will explore solar generation around the ERCOT 4CP events and compare West vs South facing solar generation during those events.

The ERCOT 4CP events are the 15-minute ERCOT grid peak events for each month in June, July, August and September.

ERCOT uses each large customer’s (including municipal utilities) total energy demand during the 4CP periods in the previous year as the basis for charges in the current year.

<p>You'll need to modify the read_csv calls in that notebook to point at these instead of the ones we've extracted and prepared for you in the /shared/JupyterHub-Examples-Data/ directory on the JupyterHub server if you would like to use the ones exported by this notebook in the analysis notebook.</p>

In [ ]:
import pandas as pd
import psycopg2
import sqlalchemy as sqla
import os
import sys
sys.path.insert(0,'..')
from config.read_config import get_database_config
%matplotlib inline
sys.executable  # shows you your path to the python you're using

In [ ]:
# read in db credentials from config/config.txt
# * make sure you add those to the config/config.txt file! *

database_config = get_database_config("../config/config.txt")


In [ ]:
# get our DB connection
engine = sqla.create_engine('postgresql://{}:{}@{}:{}/{}'.format(database_config['username'],
                                                                     database_config['password'],
                                                                     database_config['hostname'],
                                                                     database_config['port'],
                                                                     database_config['database']
                                                                     ))


In [ ]:
# These are the ERCOT 4CP events (start date/time and end date/time) for 2016 - 2019 acquired from
# http://mis.ercot.com/misapp/GetReports.do?reportTypeId=13037&reportTitle=Planned%20Service%20Four%20Coincident%20Peak%20Calculations&showHTMLView=&mimicKey

event_start_dates = ['2019-06-19 17:00:00-05', '2019-07-30 16:30:00-05', '2019-08-12 17:00:00-05', '2019-09-06 16:45:00-05',
               '2018-06-27 17:00:00-05', '2018-07-19 17:00:00-05', '2018-08-23 16:45:00-05', '2018-09-19 16:30:00-05',
               '2017-06-23 16:45:00-05', '2017-07-28 17:00:00-05', '2017-08-16 17:00:00-05', '2017-09-20 16:45:00-05',
               '2016-06-15 17:00:00-05', '2016-07-14 16:00:00-05', '2016-08-11 16:30:00-05', '2016-09-19 16:16:00-05'
              ]
event_end_dates = ['2019-06-19 17:15:00-05', '2019-07-30 16:45:00-05', '2019-08-12 17:15:00-05', '2019-09-06 17:00:00-05',
               '2018-06-27 17:15:00-05', '2018-07-19 17:15:00-05', '2018-08-23 17:00:00-05', '2018-09-19 16:45:00-05',
               '2017-06-23 17:00:00-05', '2017-07-28 17:15:00-05', '2017-08-16 17:15:00-05', '2017-09-20 17:00:00-05',
               '2016-06-15 17:15:00-05', '2016-07-14 16:15:00-05', '2016-08-11 16:45:00-05', '2016-09-19 16:31:00-05']

In [ ]:
# Select the dataids, pv direction, amount of PV of solar homes
# we're selecting homes with just South and West facing PV that have data between the first event and the last event


query = """
select dataid, pv, pv_panel_direction, total_amount_of_pv, amount_of_west_facing_pv, amount_of_south_facing_pv
from other_datasets.metadata
where pv is not null
and total_amount_of_pv is not null
and grid is not null 
and solar is not null
and pv_panel_direction in ('South', 'West')
and egauge_1min_min_time < '2016-06-15'
and egauge_1min_max_time > '2019-09-06'
LIMIT 32
"""

# create a Pandas dataframe with the data from the sql query
df = pd.read_sql_query(sqla.text(query), engine)
df

In [ ]:
# export homes to csv file
compression_opts = dict(method='zip',
                        archive_name='pv_south_vs_west_homes.zip')
df.to_csv('pv_south_vs_west_homes.zip', index=False,
          compression=compression_opts)

In [ ]:
# grab dataids and convert them to a string to put into the SQL query
dataids_list = df['dataid'].tolist()
print("{} dataids selected listed here:".format(len(dataids_list)))
dataids_str = ','.join(list(map(str, dataids_list)))
dataids_str
dataids_list

In [ ]:
# Assemble the SQL query to pull the data for the selected dataids
# 
first_start = event_start_dates.pop(0)
first_end   = event_end_dates.pop(0)
query_2 = """
select dataid, localminute, solar, grid from electricity.eg_realpower_1min 
where ((localminute >= '{}' and localminute <= '{}') """.format(first_start, first_end)

for start, end in zip(event_start_dates, event_end_dates):
    query_2 = query_2 + "OR (localminute >= '{}' and localminute <= '{}') ".format(start, end)

query_2 = query_2 + """ ) AND dataid in ({})""".format(dataids_str)

# here's what that query is
print("sql query is \n" + query_2)

# create a dataframe with the data from the sql query
df2 = pd.read_sql_query(sqla.text(query_2), engine)

# calculate usage as grid minus solar (which is actually grid + solar because solar is negative use)
# Calculate the difference with a lambda function and add it as a new column called 'usage'
df2['usage'] = df2.apply(lambda row: row.solar + row.grid, axis=1)
df2.head(15)

In [ ]:
# Total number of records in the dataset
df2['dataid'].count()

In [ ]:
# export the data to a csv file
compression_opts = dict(method='zip',
                        archive_name='pv_south_vs_west.zip')
df2.to_csv('pv_south_vs_west.zip', index=False,
          compression=compression_opts)